In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
AGGED = pd.read_csv('agg.csv', index_col = 'index')
lowlat, highlat, lowlon, highlon = AGGED["lat"]["min"], AGGED["lat"]["max"], AGGED["lon"]["min"], AGGED["lon"]["max"]

### This part is copied to ETL.py

In [3]:
def ETL(data):
    
    filtered = data.drop_duplicates().reset_index(drop = True)[['lon','lat','name']]
    filtered.to_csv("filtered.csv")

### This part is copied to process.py

In [64]:
def process(inp):
    
    start_time = time.time()
    data = pd.read_csv("filtered.csv")
    """ lower_bound_lat = data["lat"].min()
    upper_bound_lat = data["lat"].max()
    lower_bound_lon = data["lon"].min()
    upper_bound_lon = data["lon"].max()"""   
    out = []
    num = int(600 / (len(data) ** 0.5))

    for place in inp:
        
        r = 0
        by_lat = pd.DataFrame(np.zeros(shape = (0, 4))).rename(columns = {
            0: "lon", 1: "lat", 2: "name", 3: "distance"})
        by_lon = pd.DataFrame(np.zeros(shape = (0, 4))).rename(columns = {
            0: "lon", 1: "lat", 2: "name", 3: "distance"})

        while True:
            
            r += num
            
            if not by_lat.empty:
                lower_lat = by_lat.loc[0]["lat"]
                upper_lat = by_lat.loc[len(by_lat) - 1]["lat"]
                new_lat = data.loc[(abs(data["lat"] - place["lat"]) <= r) & ((data["lat"] < lower_lat) | \
                                        (data["lat"] > upper_lat))]
                new_lat["distance"] = (place['lon'] - new_lat['lon']) ** 2 + (place['lat'] - new_lat['lat']) ** 2
                by_lat = pd.concat([by_lat, new_lat], ignore_index = True).sort_values("lat").reset_index(drop = True)
                
            else:
                by_lat = pd.concat([by_lat, data.loc[abs(data["lat"] - place["lat"]) <= r].sort_values("lat").reset_index(
                    drop = True)])
                by_lat["distance"] = (place['lon'] - by_lat['lon']) ** 2 + (place['lat'] - by_lat['lat']) ** 2
                
            if not by_lon.empty:
                lower_lon = by_lon.loc[0]["lon"]
                upper_lon = by_lon.loc[len(by_lon) - 1]["lon"]
                new_lon = data.loc[(abs(data["lon"] - place["lon"]) <= r) & ((data["lon"] < lower_lon) | \
                                        (data["lon"] > upper_lon))]
                new_lon["distance"] = (place['lon'] - new_lon['lon']) ** 2 + (place['lat'] - new_lon['lat']) ** 2
                by_lon = pd.concat([by_lon, new_lon], ignore_index = True).sort_values("lon").reset_index(drop = True)
                
            else:
                by_lon = pd.concat([by_lon, data.loc[abs(data["lon"] - place["lon"]) <= r].sort_values("lon").reset_index(
                    drop = True)])
                by_lon["distance"] = (place['lon'] - by_lon['lon']) ** 2 + (place['lat'] - by_lon['lat']) ** 2        
                
            if by_lon.empty or by_lat.empty:
                continue

            lat_min = by_lat.iloc[[by_lat["distance"].idxmin()]].reset_index(drop = True)
            lon_min = by_lon.iloc[[by_lon["distance"].idxmin()]].reset_index(drop = True)
            
            if lat_min.equals(lon_min):
                closest_place = {'lon': lat_min.loc[0, 'lon'], 'lat': lat_min.loc[0, 'lat'],
                         'name': lat_min.loc[0, "name"]}
                out.append(closest_place.copy())
                break
                
            """if ((place["lat"] - r < lower_bound_lat) and (place["lat"] + r > upper_bound_lat)) and \
            ((place["lon"] - r < lower_bound_lon) and (place["lon"] + r < upper_bound_lon)):
                print("r:", r)
                break"""
               
    calc_time = time.time() - start_time
    
    return out, calc_time

### Now it's time to test your solution

In [5]:
def generate_input(input_size):
    
    return [{c: np.random.uniform(AGGED.loc['min', c], AGGED.loc['max', c]) for c in AGGED.columns}
         for _ in range(input_size)]

In [65]:
MINI_DATA = [12e3]#[3e3,6e3,
SMALL_DATA = [3e4,6e4,30e4,60e4]
BIG_DATA = [3e6,6e6,12e6,24e6,36e6,48e6,60e6]

use_datasets = MINI_DATA
data_dicts = {int(data_size): pd.read_csv('../data_sources/hotels_data/hotels-%d.csv' % data_size)
              for data_size in use_datasets}

In [66]:
input_sizes = [1, 5] #, 10, 20, 200, 1000]
input_set = {}

for data_name in data_dicts.keys():
    input_set[data_name] = {}
    for input_size in input_sizes:
        input_set[data_name][input_size] = generate_input(input_size)

In [67]:
solution_dict = {}

for data in input_set.keys():

    print("---------------------------\nData size: ", data)

    ETL(data_dicts[data])    
    solution_dict[data] = {}

    for inp in input_set[data].keys():

        print("---------------------------\n   Input size: ", inp)

        solution, calc_time = process(input_set[data][inp])
        solution_dict[data][inp] = {"solution": solution, "calc_time": calc_time}

        print("\n   Closest hotel(s): ", solution)
        print("\n   Calculation time: %f seconds" %round(calc_time, 2))

---------------------------
Data size:  12000
---------------------------
   Input size:  1

   Closest hotel(s):  [{'lon': -106.41911437451729, 'lat': 31.78826497758857, 'name': 'Hilton Garden Inn El Paso / University - rbnb 48'}]

   Calculation time: 0.170000 seconds
---------------------------
   Input size:  5


C:\Hanga\programok\PYTHON\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Hanga\programok\PYTHON\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Hanga\programok\PYTHON\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


   Closest hotel(s):  [{'lon': -73.01993968666878, 'lat': -41.432208422672616, 'name': 'Antupiren - rbnb 78'}, {'lon': -72.51042493031153, 'lat': 46.36927245485092, 'name': 'Motel Le Deauville - rbnb 40'}, {'lon': -21.94900687114935, 'lat': 64.11926026363804, 'name': 'Apartment K - rbnb 57'}, {'lon': 107.02450597527302, 'lat': 47.89123000269159, 'name': 'Best Western Premier Tuushin Hotel - rbnb 29'}, {'lon': -41.88735942999581, 'lat': -22.746907606123965, 'name': 'Pousada Barlavento - rbnb 37'}]

   Calculation time: 1.560000 seconds
